# Breadth First Search

We will be trying to find and visualize the path between Equestrian Statue of Edward VII and Bahen Center of Technology around Toronto University campus using breadth first search


__BREADTH-FIRST-SEARCH__ ( _source_ , _destination_ ) __return__ a route   
&emsp;_frontier_ &larr; a **FIFO** initialized with _source_ node  
&emsp;_explored_ &larr; _empty_    
&emsp;_found_ &larr; _False_  
&emsp;__while__  _frontier_ __is not__ _empty_ __and__ _found_ __is__ _False_ __do__  
&emsp;&emsp;&emsp;_node_ &larr; _frontier_.pop()   
&emsp;&emsp;&emsp;__add__ _node_ __to__ _explored_    
&emsp;&emsp;&emsp;__for__ _child_ __in__ _node_.expand() __do__   
&emsp;&emsp;&emsp;&emsp;&emsp;__if__ _child_ __is not in__ _explored_ __and__ _child_ __is not in__ _frontier_   __then__  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; __if__ _child_ __is__ _destination_ __then__  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; _route_ &larr; _child_.route()  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; _found_ &larr; _True_  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;__add__ _child_ __to__ _frontier_    
&emsp;__return__  _route_

In [1]:
import osmnx as ox
import time
from tqdm import tqdm
from collections import deque
from utilities import *

Here OSMnx find the largest connected component centered around the ```location point``` with specified ```dist``` on each side (w/n/e/s).

In [2]:
location_point = (43.661667, -79.395)
G = ox.graph_from_point(location_point, dist=300, clean_periphery=True, simplify=True)

Exception: Server returned
<Response [405]> Method Not Allowed
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"/>
<title>405 - HTTP verb used to access this page is not allowed.</title>
<style type="text/css">
<!--
body{margin:0;font-size:.7em;font-family:Verdana, Arial, Helvetica, sans-serif;background:#EEEEEE;}
fieldset{padding:0 15px 10px 15px;} 
h1{font-size:2.4em;margin:0;color:#FFF;}
h2{font-size:1.7em;margin:0;color:#CC0000;} 
h3{font-size:1.2em;margin:10px 0 0 0;color:#000000;} 
#header{width:96%;margin:0 0 0 0;padding:6px 2% 6px 2%;font-family:"trebuchet MS", Verdana, sans-serif;color:#FFF;
background-color:#555555;}
#content{margin:0 0 0 2%;position:relative;}
.content-container{background:#FFF;width:96%;margin-top:8px;padding:10px;position:relative;}
-->
</style>
</head>
<bod

Here you need to specify which node from our graph is the source (Equestrian Statue of Edward VII) and which is the destination node (Bahen Center of Technology). You can do so by acquiring the decimal coordinates of the desired node and use [```osmnx.distance.get_nearest_node```](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.get_nearest_node) method

I used the aforementioned method and found that the osmid of the nodes for destination and source are 389677909, 55808290 respectively

In [ ]:
highlighted = [389677909, 55808290]

# marking both the source and destination node

nc = ['r' if node in highlighted else '#336699' for node in G.nodes()]
ns = [50 if node in highlighted else 8 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2)

In [ ]:
draw_map(G, highlight = highlighted)

Each node in our graph is represented as a dictionary with many attributes of no interest to us now, so manipulating them would obfuscate the algorithm jumping through hoops to get only one attribute from the dictionary. <b>so</b> we define class ```Node``` which only retains the data we need to be able to do searching and traversing, like the parent of the node (the one that produced it from its expansion) and the length of the edge between the parent and the node itself.

Please check its source code in ```./utilities/src/common.py``` to know how it captures the data from the graph. 

In [ ]:
source(Node)

# The Algorithm

In [ ]:
# first define the origin/source nodes as Node
origin = Node(graph = G, osmid = 55808290)
destination = Node(graph = G, osmid = 389677909)

In [ ]:
%%time
bar = tqdm(total=len(G))

# this is where we would save our result at the end
route = []

# we will be using deque instead of normal python lists
# to have O(1) [pop/append]-ing instead of O(n) in lists
# because deques are implemented with linked-list and
# lists are implemented with arrays 

frontier = deque([origin])

# this is a directed multigraph so we need to have memory
# so we don't get stuck in a loop
explored = set()

found = False

while frontier and not found:
    bar.update(1); time.sleep(0.05) # for the progress bar -- ignore

    node = frontier.popleft()
    explored.add(node)
    for child in node.expand():
        if child not in explored and child not in frontier:
            if child == destination:
                route = child.path()
                found = True
            frontier.append(child)

bar.close()
print("The route is \n\n",route, "\n\nits cost is\n\n", cost(G, route))

In [ ]:
fig, ax = ox.plot_graph_route(G, route)

In [ ]:
draw_route(G, route)

We will be using uber's [kepler.gl](https://kepler.gl/) for one time in this notebook only to make you aware of it, but unfortunatley we won't be using it because of its of its limited API.

In [ ]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd

In [ ]:
map1 = KeplerGl(height=600, width=800)
map1

Kepler is not integrated with osmnx so we need to build the GeoDataFrame of the route data by ourselves. And the above widget will change accordingly, so here we go.  

The is what we meant by limited API.

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

First let's make a dataframe for the nodes of the route -- this dataframe need to have the y (latitude), x (longitude) of these nodes. That is done by filtering the nodes GeoDataFrame by only keeping the rows which have id in the route list. 

In [ ]:
nodes

In [ ]:
route_data_frame = nodes.loc[nodes.index.isin(route)]

We will need to load the dataframe into the map as what we will show in the next couple of cells, after that we need to use the GUI of the widget to add it manually.

In [ ]:
route_data_frame

In [ ]:
map1.add_data(data=route_data_frame , name="route")

# if you run this cell again, it will fail
# that's what we meant when we said
# the binding of jupyter to JS is very primitive

Okay now we have update the widget with the nodes data frame and they should be displayed on the widget above now.

The hard part now is to make them connected and draw the lines.

What we have made above is called adding a layer to the visualization, and every layer need certain attributes to be rendered correctly.  

The "nodes" layers is called point layers which needs a dataframe consists of a rows of coordinates for each node/point to rendered. To display the route, we need use layer called "lines" layer which needs to have the source and destination coordinates for each line/road and after we add this dataframe, we need to add it manually.  

Let's get started

In [ ]:
# let's build the dictionary from which
# we can convert it to pandas dataframe

# list of tuples (latitude, longitude)
src_nodes = []
dest_nodes = []

for src_node_edge, dest_node_edge in zip(route, route[1:]):
    
    src_nodes.append(
                    (
                     float(nodes.loc[[src_node_edge]]['x']),
                     float(nodes.loc[[src_node_edge]]['y'])
                    )
                   )
    
    dest_nodes.append(
                    (
                     float(nodes.loc[[dest_node_edge]]['x']),
                     float(nodes.loc[[dest_node_edge]]['y'])
                    )
                   )


In [ ]:
data_frame = {
    'source_x' : [coordinates[0] for coordinates in src_nodes],
    'dest_x' :   [coordinates[0] for coordinates in dest_nodes],
    'source_y' : [coordinates[1] for coordinates in src_nodes],
    'dest_y' :   [coordinates[1] for coordinates in dest_nodes],
}

In [ ]:
edges_route = pd.DataFrame(data_frame, columns = ['source_x', 'dest_x', 'source_y', 'dest_y'])


In [ ]:
edges_route # y is latitude and x is longitude

Okay let's have a clean map and add these lines step by step.

In [ ]:
map2 = KeplerGl(height=600, width=800)
map2

In [ ]:
map2.add_data(data=edges_route , name="route")

Okay now you need to press the little arrow icon on the upper left which opens a toolset to add layers, filters, and other things.

It should open on the "Layers" tab automatically, after that press **Add Layer** and name the layer anything and then from **Basic** dropbox choose **Line**, it will prompt you to specify Source Lat/Lng and Target Lat/Lng. Choose "source_y" in Source Lat and "source_x" in Source Lng, and then choose "dest_y" in Target Lat and "dest_x" in Target Lng.  

The "route" should be in its place now, to show it you need to scroll through the map of the world above to UofT coordinates and you should see the route in its place.

You can add another layer to see the nodes of the route in the same visualization by adding "Point" layer and remember that y is latitude and x in longitude.

---

This seems like a lot of hustle, but we wanted to introduce to you this tool because it is Uber :) and the project is [very active](https://github.com/keplergl/kepler.gl). 

The jupyter bindings are still primitive and needs a lot of work to be on par with leaflet-based maps, but the tool is originally intended to be a React Component and some people did some very slick visualization with it which you can check them on the kepler.gl website and [here](https://github.com/uber-web/kepler.gl-data). So keep this tool in mind when you are working on the next Uber :))